Use PyTorch, GPU and Colab.

Implement a simple convolutional neural network with training and validation on CIFAR10 dataset, limited to subset of 3k train and 3k test images.
Train the neural network for ~30 epochs with Adam optimizer and batch size equal to 64. Implement validation on batches.
  
Code the following incrementally:
- Prevent overfitting using a) dropout and b) data augmentation.
- Add test augmentation.
- For better results use transfer learning.

In [ ]:
import torch
torch.manual_seed(0)
 
import random
random.seed(0)
 
import numpy as np
np.random.seed(0)
 
import matplotlib.pyplot as plt
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

transform = transforms.Compose( [ transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) ] )
 
batch_size = 64
num_workers = 0
path = '~/Downloads/CIFAR10/'
 
def create_loader(path: str, train: bool, transform) -> torch.utils.data.DataLoader:
  # TODO
 
transform_tr =  transform
transform_val = transform
 
trainloader = create_loader(path, train=True, transform=transform_tr)
testloader = create_loader(path, train=False, transform=transform_val)

In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# sample data
display_limit = 8
 
def imshow(img):
  img = img / 2 + 0.5     # unnormalize
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg, (1, 2, 0)))
  plt.show()
 
# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()
 
# show images
imshow(torchvision.utils.make_grid(images[:display_limit]))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(display_limit)))

In [ ]:
W, H = 32, 32
Ch = 3
 
# should have conv 32, conv 64, conv 128 (all with kernel size 3x3, stride 2 and padding 1) with relu activations, hidden linear layer 40 with relu, and linear output
class Net(nn.Module): 
 def __init__(self, init_W, init_H, init_Ch, linear_constructor):
   self._linear_constructor = linear_constructor
   super(Net, self).__init__()
	# TODO

In [ ]:
 # Use Adam optimizer

In [ ]:
from torch.utils.tensorboard import SummaryWriter
 
epochs = 40
 
def train(epochs, trainloader, testloader, net):
  writer = SummaryWriter()
  
  for epoch in range(epochs):  # loop over the dataset multiple times
    # TODO

In [ ]:
def save_net(path, net):
  # TODO
 
PATH = './cifar_net.pth'
save_net(PATH, net)

In [ ]:
def load_params(path, net):
  # TODO

load_params(PATH, net)

In [ ]:
def validate(loader, name):
  #TODO
 
validate(trainloader, "train")
validate(testloader, "test")

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir runs